In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import copy
import ipywidgets as iwdg
import IPython.display as idsp

In [ ]:
plotFont = {
    'family' : 'sans-serif',
    'weight' : 'normal',
    'size'   : 12
}
matplotlib.rc('font', **plotFont)
plt.rc('legend', fontsize=10)
defaultColorCycle = plt.rcParams["axes.prop_cycle"]

In [ ]:
plotDefsBase = [
    {
        'varName1': 'x', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'x',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'x', 'varName2': 'xp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'y', 'varName2': 'yp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'z', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'Ekin',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
]

In [ ]:
distrDatabase = {
    'CTSB-N02-F100-E06-S0.5-T5.0_FCTest_Pavel_SolC_CLICTW-OptionA08B7-Bc0.50': {
        'plotDefs': [
            {
                'lims1': (-25, 25.),   # [mm]
                'lims2': (-25., 25.)   # [mm]
            },
            {
                'lims1': (62.2, 62.5),   # [ns]
            },
            {},
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims1': (9999., 10001.),   # [mm]
                'lims2': (0., 400.)   # [MeV/c]
            },
            {},
            {},
        ]
    },
    'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50': {
        'plotDefs': [
            {
                'lims1': (-25, 25.),   # [mm]
                'lims2': (-25., 25.)   # [mm]
            },
            {
                'lims1': (62.8, 63.1),   # [ns]
            },
            {},
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims1': (9999., 10001.),   # [mm]
                'lims2': (0., 400.)   # [MeV/c]
            },
            {},
            {},
        ]
    },
    'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50': {
        'plotDefs': [
            {
                'lims1': (-25, 25.),   # [mm]
                'lims2': (-25., 25.)   # [mm]
            },
            {
                'lims1': (58.6, 58.9),   # [ns]
            },
            {},
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims2': (-30., 30.),   # [mrad]
            },
            {
                'lims1': (9999., 10001.),   # [mm]
                'lims2': (0., 400.)   # [MeV/c]
            },
            {},
            {},
        ]
    }
}

for distr in distrDatabase.values():
    distr['plotDefs'][1]['lims2'] = distr['plotDefs'][0]['lims1']   # [mm]
    distr['plotDefs'][2]['lims1'] = distr['plotDefs'][1]['lims1']   # [ns]
    distr['plotDefs'][2]['lims2'] = distr['plotDefs'][0]['lims2']   # [mm]
    distr['plotDefs'][3]['lims1'] = distr['plotDefs'][0]['lims1']   # [mm]
    distr['plotDefs'][4]['lims1'] = distr['plotDefs'][0]['lims2']   # [mm]
    distr['plotDefs'][6]['lims1'] = distr['plotDefs'][1]['lims1']   # [ns]
    distr['plotDefs'][6]['lims2'] = distr['plotDefs'][5]['lims2']   # [MeV]
    distr['plotDefs'][7]['lims1'] = distr['plotDefs'][6]['lims1']   # [ns]
    distr['plotDefs'][7]['lims2'] = distr['plotDefs'][5]['lims2']   # [MeV/c]

In [ ]:
def load_distr(distrName):
    sdfFilePath = './BeamDistrs/Positrons_200MeV_Yongke/' + distrName + '.dat.sdf_txt'
    distr = bd.load_standard_fwf(sdfFilePath)
    return distr

# FCC-ee Injector Beams App

In [ ]:
distrKeys = list(distrDatabase.keys())
distr_dropdown = iwdg.Dropdown(
    options=distrKeys,
    value=distrKeys[-1],
    description='Distribution:',
    disabled=False,
    layout={'width': 'initial'},
)

plot_out = []
for pDef in plotDefsBase:
    plot_out.append(iwdg.Output())
descr_out = iwdg.Output(overflow='auto')
descr_out.layout.width = '1024px'

def plot_all(distr, distrName):
    for p_out in plot_out:
        p_out.clear_output()
    descr_out.clear_output()
    plotDefs = [{**pDefBase, **pDefDb} for pDefBase, pDefDb in zip(plotDefsBase, distrDatabase[distrName]['plotDefs'])]
    for pDef, p_out in zip(plotDefs, plot_out):
        with p_out:
            bd.plot_distr([distr], [pDef], figHeight=7, figWidth=12.44)
            plt.show()
    with descr_out:
        idsp.display(distr.describe())
        
def distr_dropdown_eventhandler(change):
    distr = load_distr(change.new)
    filterSpecs = {
        'x': distrDatabase[change.new]['plotDefs'][3]['lims1'],
        'xp': distrDatabase[change.new]['plotDefs'][3]['lims2'],
        'y': distrDatabase[change.new]['plotDefs'][4]['lims1'],
        'yp': distrDatabase[change.new]['plotDefs'][4]['lims2'],
        't': distrDatabase[change.new]['plotDefs'][7]['lims1'],
        'pz': distrDatabase[change.new]['plotDefs'][7]['lims2'],
    }
    distr = bd.filter_distr(distr, filterSpecs)
    plot_all(distr, change.new)
    
distr_dropdown.observe(distr_dropdown_eventhandler, names='value')
tab_layout = iwdg.Layout(margin='25px 0 0 0')
outputs_tab = iwdg.Tab(plot_out+[descr_out], layout=tab_layout)
for tabInd, pDef in enumerate(plotDefsBase):
    outputs_tab.set_title(tabInd, pDef['varName1']+'-'+pDef['varName2'])
outputs_tab.set_title(len(plotDefsBase), 'Description')
dashboard = iwdg.VBox([distr_dropdown, outputs_tab])
display(dashboard)
distr_dropdown.value = distrKeys[0]